# Final Project Phase 3 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 3 of the Final Project. Complete all sections below as specified in the instructions for the project, covering all necessary details. We will use this to grade your individual code (Do this whether you are in a group or not). Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable. 
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or 
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning


Transfer/update the data collection and cleaning you created for Phase II below. You may include additional cleaning functions if you have extra datasets. If no changes are necessary, simply copy and paste your phase II parsing/cleaning functions.


## Downloaded Dataset Requirement



In [2]:
## We separated the two csv files. One that contains raptor_elo, and one without. (This one is without)
## We are creating two different files because we aren't sure whether or not the next phase will require the use 
## of this particular column.
import numpy as np
import pandas as pd
def data_parser():
    df = pd.read_csv("nba_elo.csv", delimiter = ",")
    df = df.drop(df[df["season"]<1994].index, axis = 0) # our other datasets only go to 1994
    ##"raptor1_pre","raptor2_pre","raptor_prob1","ra"
    df = df.drop(["date", "neutral","elo1_pre","elo2_pre","elo_prob1","elo_prob2","raptor2_pre","raptor1_pre", "carm-elo_prob1", "carm-elo_prob2", "carm-elo1_post", "carm-elo2_post", "carm-elo1_pre", "carm-elo2_pre", "raptor_prob1", "raptor_prob2"], axis = 1) # removing nonrelevant data columns
    df["playoff"] =  1 if df.empty else 0
    table1 = df.drop(["team2", "elo2_post", "score2"], axis = 1)
    
    table2 = df.drop(["team1", "elo1_post", "score1"], axis = 1)
    table2 = table2.rename(columns = {"team2": "team1", "elo2_post":"elo1_post", "score2": "score1"})
    newdf = pd.concat([table1,table2], axis = 0)
    newdf = newdf.rename(columns = {"team1" : "teams", "elo1_post" : "team_elos","score1":"game_scores"})
    newdf.insert(1, "conference", np.where((newdf["teams"] == "BOS")|(newdf["teams"]=="CHI")|(newdf["teams"]=="ATL")|(newdf["teams"]=="BKN")|(newdf["teams"]=="CLE")|(newdf["teams"]=="NYK")|(newdf["teams"]=="DET")|(newdf["teams"]=="MIA")|(newdf["teams"]=="PHI")|(newdf["teams"]=="IND")|(newdf["teams"]=="ORL")|(newdf["teams"]=="TOR")|(newdf["teams"]=="MIL")|(newdf["teams"]=="WAS")| (newdf["teams"] == "NJN"), 1, 0))
    newdf = newdf.groupby(["season", "teams"]).mean()
    newdf = newdf.drop("playoff", axis = 1)
    newdf["conference"] = np.where(newdf["conference"] == 1, "E", "W")
    newdf.to_csv('nba_without.csv')
    
    return newdf

    

############ Function Call ############
data_parser()

conference    team_elos  game_scores
season teams                                     
1994   ATL            E  1601.365422   100.129032
       BOS            E  1444.515178   100.817073
       CHH            W  1491.782755   106.487805
       CHI            E  1616.152185    97.489130
       CLE            E  1580.306072   100.941176
...                 ...          ...          ...
2021   SAC            W  1439.366501   113.736111
       SAS            W  1499.585256   110.876712
       TOR            E  1540.286579   111.263889
       UTA            W  1651.818171   116.807692
       WAS            E  1423.886133   116.329114

[825 rows x 3 columns]

## Web Collection Requirement \#1


In [4]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import re
import pandas as pd
import numpy as np
def web_parser1():
    #make these into the col names and add to empty df
    startdf = pd.DataFrame()
    
    for year in range(1994, 2022):
        count = 0
        response = requests.get(f"http://www.espn.com/nba/attendance/_/year/{year}")
        soup = BeautifulSoup(response.text, "html.parser")
        tabledata = soup.find("div", {"class": "col-main"})
        rows = tabledata.find_all("tr")
        newrows = []
        for row in rows[2:]:
            row = row.find_all("td")
            newrow = []
            for cell in row:
                newrow.append(cell.text)
            # stops adding rows once it reaches the extraneous stats at the bottom of the table
            if int(row[2].text) < 2:
                break
            newrows.append(newrow)
            count += 1
        # column headers for the data
        df = pd.DataFrame(newrows, columns = ["Rank", "Team", "Home Games", "Total Home Attendance", "Average Home Attendance", "Home Percent Capacity", "Road Games", "Average Road Attendance", "Road Percent Capacity", "Overall Games", "Overall Average Attendance", "Overall Percent Capacity"])
        df.insert(0, "Season", [int(year) for i in range(df.count()[0])])
        df.insert(1, "Conference", np.where((df["Team"] == "Celtics")|(df["Team"]=="Bulls")|(df["Team"]=="Hawks")|(df["Team"]=="Nets")|(df["Team"]=="Cavaliers")|(df["Team"]=="NY Knicks")|(df["Team"]=="Pistons")|(df["Team"]=="Heat")|(df["Team"]=="76ers")|(df["Team"]=="Pacers")|(df["Team"]=="Magic")|(df["Team"]=="Raptors")|(df["Team"]=="Bucks")|(df["Team"]=="Wizards")|(df["Team"]=="Hornets"), "E", "W"))
        df = df.drop(["Home Games", "Road Games", "Overall Games"], axis = 1)
        startdf = pd.concat((startdf, df), axis = 0)
    startdf = startdf.set_index("Season")
    startdf.to_csv("nba_attendance.csv")
    return startdf





############ Function Call ############
web_parser1()


Conference Rank           Team Total Home Attendance  \
Season                                                        
1994            W    1       Pelicans               971,618   
1994            W    2          Spurs               904,167   
1994            W    3           Jazz               814,502   
1994            E    4      NY Knicks               810,283   
1994            E    5        Pistons               806,641   
...           ...  ...            ...                   ...   
2021            E                Heat                     0   
2021            W       Trail Blazers                     0   
2021            W               Kings                     0   
2021            W             Thunder                     0   
2021            E             Wizards                     0   

       Average Home Attendance Home Percent Capacity Average Road Attendance  \
Season                                                                         
1994                    23,698                    --                  16,977   
1994                    22,052                    --                  16,260   
1994                    19,865                    --                  16,441   
1994                    19,763                    --                  16,868   
1994                    19,674                    --                  15,532   
...                        ...                   ...                     ...   
2021                         0                    --                   3,115   
2021                         0                    --                   2,139   
2021                         0                    --                   2,469   
2021                         0                    --                   1,946   
2021                         0                    --                   1,467   

       Road Percent Capacity Overall Average Attendance  \
Season                                                    
1994                      --                     20,337   
1994                      --                     19,156   
1994                      --                     18,153   
1994                      --                     18,315   
1994                      --                     17,603   
...                      ...                        ...   
2021                    16.8                      2,769   
2021                    11.5                          0   
2021                    13.4                      2,033   
2021                    10.6                      1,646   
2021                     7.8                      1,320   

       Overall Percent Capacity  
Season                           
1994                         --  
1994                         --  
1994                         --  
1994                         --  
1994                         --  
...                         ...  
2021                       14.9  
2021                       11.5  
2021                       11.1  
2021                        9.0  
2021                        7.0  

[825 rows x 10 columns]

## Web Collection Requirement #2

In [5]:
## This code works but sometimes reaches a read timed out error if one requests too many times in succession. 
from nba_api.stats.endpoints import leaguedashteamstats
import requests
import pandas as pd
import re
import numpy as np
# nba_teams = teams.get_teams()
# headers = response.get_dict()["resultSets"][0]["headers"]
def web_parser2():
    panda = pd.DataFrame()
    for x in range (1996,2020):
        if x >= 2009:
            response = leaguedashteamstats.LeagueDashTeamStats(
            team_id_nullable= '0',
            season= str(x) + "-" + str(x + 1 - 2000))
        elif x >= 1999:
            response = leaguedashteamstats.LeagueDashTeamStats(
            team_id_nullable= '0',
            season= str(x) + "-0" + str(x + 1 - 2000))
        else:
            response = leaguedashteamstats.LeagueDashTeamStats(
            team_id_nullable= '0',
            season= str(x) + "-" + str(x + 1 - 1900))
        headers = response.get_dict()["resultSets"][0]["headers"]
        teams = column = response.get_dict()["resultSets"][0]["rowSet"]
        team = response.get_dict()["resultSets"][0]["rowSet"][0]
        mydict = {}
        for i in range(len(teams)):
            team = response.get_dict()["resultSets"][0]["rowSet"][i]
            mydict[str(x) + " " + team[1]] = ([team[x] for x in range(2,5)])
        df = pd.DataFrame.from_dict(mydict,orient = "index",columns = ["GP","W","L"])
        df.insert(0, "Season", [re.match("(\d+) ", i).group() for i in df.index.values])
        df.insert(1, "Team", [re.search("\d+ (\w+ \w+)", i).group(1) for i in df.index.values])
        df = df.set_index("Season")
        #df = df.drop("Season", axis = 1)
        df.insert(1, "Conference", np.where((df["Team"] == "Boston Celtics")|(df["Team"]=="Chicago Bulls")|(df["Team"]=="Atlanta Hawks")|(df["Team"]=="Brooklyn Nets")|(df["Team"]=="Cleveland Cavaliers")|(df["Team"]=="New York")|(df["Team"]=="Detroit Pistons")|(df["Team"]=="Miami Heat")|(df["Team"]=="Philadelphia 76ers")|(df["Team"]=="Indiana Pacers")|(df["Team"]=="Orlando Magic")|(df["Team"]=="Toronto Raptors")|(df["Team"]=="Milwaukee Bucks")|(df["Team"]=="Washington Wizards")|(df["Team"]=="Charlotte Hornets")|(df["Team"] == "New Jersey"), "E", "W"))
        panda = pd.concat([panda,df],axis = 0)
    panda.to_csv("nba_record.csv")
    return panda




############ Function Call ############
web_parser2()


Team Conference  GP   W   L
Season                                            
1996          Atlanta Hawks          E  82  56  26
1996         Boston Celtics          E  82  15  67
1996      Charlotte Hornets          E  82  54  28
1996          Chicago Bulls          E  82  69  13
1996    Cleveland Cavaliers          E  82  42  40
...                     ...        ...  ..  ..  ..
2019       Sacramento Kings          W  72  31  41
2019            San Antonio          W  71  32  39
2019        Toronto Raptors          E  72  53  19
2019              Utah Jazz          W  72  44  28
2019     Washington Wizards          E  72  25  47

[712 rows x 5 columns]

#Inconsistency Revisions
 **If you were requested to revise your inconsistency section from Phase II, enter your responses here. Otherwise, ignore this section.**

For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. 

2. 

3. 

4. (if applicable)

5. (if applicable)


## Data Sources

Include sources (as links) to your datasets. If any of these are different from your sources used in Phase II, please <b>clearly</b> specify.

*   Downloaded Dataset Source:
*   Web Collection #1 Source:
*   Web Collection #2 Source:



# Data Analysis
For the Data Analysis section, you are required to utilize your data to complete the following:

*   Create at least 5 insights
*   Generate at least 3 data visualizations
*   Export aggregated data to at least 1 summary file 

Create a function for each of the following sections mentioned above. Do not forget to fill out the explanation section for each function. 

Make sure your data analysis is not too simple. Performing complex aggregation and using modules not taught in class shows effort, which will increase the chance of receiving full credit. 

# Graphical User Interface (GUI) Implementation
If you decide to create a GUI for Phase II, please create a separate Python file (.py) to build your GUI. You must submit both the completed PhaseII.ipynb and your Python GUI file.

## Insights

In [6]:
def insight1():
    df1 = data_parser()
    df1 = df1.drop([1994, 1995, 2020, 2021], axis=0)
    df1 = df1.reset_index()
    df1 = df1.rename(columns={"season":"Season", "conference":"Conference"})
    df1 = df1.groupby(["Season","Conference"]).mean()
    df1 = df1.sort_index(axis=0)
    df2 = pd.read_csv("nba_record.csv")
    df2 = df2.drop(df2.index[234])
    df2 = df2.groupby(["Season","Conference"]).mean()
    df2 = df2.sort_index(axis=0)
    newdf = pd.concat((df1, df2), axis=1)
    newdf = newdf.drop(["game_scores", "GP"], axis=1)
    return newdf





############ Function Call ############
insight1()

team_elos          W          L
Season Conference                                   
1996   E           1484.265598  43.071429  38.928571
       W           1490.253428  39.066667  42.933333
1997   E           1512.029824  43.200000  38.800000
       W           1476.325778  38.642857  43.357143
1998   E           1521.665809  25.266667  24.733333
       W           1472.575446  24.714286  25.285714
1999   E           1521.241145  39.866667  42.133333
       W           1478.499710  42.214286  39.785714
2000   E           1493.815316  37.733333  44.266667
       W           1506.101020  44.500000  37.500000
2001   E           1465.883489  39.533333  42.466667
       W           1535.524632  42.571429  39.428571
2002   E           1471.184157  37.714286  44.285714
       W           1533.728050  44.066667  37.933333
2003   E           1472.921760  37.000000  45.000000
       W           1533.101246  44.733333  37.266667
2004   E           1461.793721  40.428571  41.571429
       W           1545.657767  43.066667  38.933333
2005   E           1474.849168  40.142857  41.857143
       W           1519.056437  41.750000  40.250000
2006   E           1484.644090  39.285714  42.714286
       W           1512.451947  42.500000  39.500000
2007   E           1483.967741  39.285714  42.714286
       W           1518.167357  42.500000  39.500000
2008   E           1483.126653  41.857143  40.142857
       W           1522.019277  40.250000  41.750000
2009   E           1505.387935  39.285714  42.714286
       W           1502.380280  42.500000  39.500000
2010   E           1492.767207  38.928571  43.071429
       W           1515.208766  42.812500  39.187500
2011   E           1491.902305  33.357143  32.642857
       W           1516.895332  32.687500  33.312500
2012   E           1494.147608  39.714286  42.142857
       W           1516.250502  42.062500  39.937500
2013   E           1490.010777  36.642857  45.357143
       W           1517.330508  44.812500  37.187500
2014   E           1460.701795  38.466667  43.533333
       W           1538.887929  43.533333  38.466667
2015   E           1468.438832  40.533333  41.466667
       W           1536.733208  41.466667  40.533333
2016   E           1496.319753  39.600000  42.400000
       W           1514.114638  42.400000  39.600000
2017   E           1498.957851  40.200000  41.800000
       W           1510.874003  41.800000  40.200000
2018   E           1502.228543  39.200000  42.800000
       W           1509.819126  42.800000  39.200000
2019   E           1487.159280  33.800000  36.000000
       W           1522.127515  36.800000  34.600000

### Insight 1 Explanation

Someone may be interested in comparing the performance of the Eastern vs. Western conferences for each season using both the elo metric from our data parser and wins and losses from web parser 2. These two measures of performance should be correlated but may not be exactly the same so using both may give more insight into the performance of each conference. It would make sense for someone to be interested in finding out which conference is better overall.

In [7]:
def insight2():
    team_dict = {"ATL" : "Hawks","BOS" : "Celtics","Van":"Raptors","BRK": "Nets","WAS":"Wizards","CHH" : "Hornets","CHO":"Hornets","CHI" : "Bulls", "CLE" : "Cavaliers","DAL" : "Mavericks","DEN" : "Nuggets","DET": "Pistons","GSW" : "Warriors","HOU" : "Rockets","IND" : "Pacers","LAC" : "Clippers","LAL" : "Lakers","MEM":"Grizzlies","MIA" : "Heat","MIN":"Timberwolves","MIL": "Bucks","NJN": "Nets","BKN" : "Nets","NYK" : "NY Knicks","ORL" : "Magic","PHI" :"76ers","PHO" : "Suns","POR":"Trailblazers","SAC":"Kings","SAS" : "Spurs","SEA": "Thunder","OKC" :"Thunder", "UTA" : "Jazz", "WSB" : "Wizards","NOP" : "Pelicans","TOR":"Raptors"}
    elo = pd.read_csv("nba_without.csv")
    elo = elo.reset_index()
    elo = elo.drop(["index"],axis = 1)
    elo = elo.replace({"teams":team_dict})
    attendance = pd.read_csv("nba_attendance.csv", thousands=',')
    attendance["Overall Average Attendance"].values.astype(int)
    attendance = attendance.sort_values(["Season", "Team"])
    attendance = attendance.drop(["Rank", "Total Home Attendance", "Average Home Attendance", "Home Percent Capacity", "Average Road Attendance", "Road Percent Capacity", "Overall Percent Capacity", "Conference", "Team"], axis=1)
    attendance = attendance.set_index("Season")
    elo = elo.sort_values(["season", "teams"])
    elo = elo.rename(columns={"season":"Season"})
    elo = elo.set_index("Season")
    final = pd.concat((elo, attendance), axis=1)
    final = final.drop("game_scores", axis=1)
    
    return final





############ Function Call ############
insight2()

teams conference    team_elos  Overall Average Attendance
Season                                                                  
1994           76ers          E  1381.289578                       14343
1994           Bucks          E  1344.409726                       15575
1994           Bulls          E  1616.152185                       18327
1994       Cavaliers          E  1580.306072                       17035
1994         Celtics          E  1444.515178                       15420
...              ...        ...          ...                         ...
2021         Thunder          W  1402.002093                        1646
2021    Timberwolves          W  1346.175147                        1996
2021    Trailblazers          W  1528.737374                           0
2021        Warriors          W  1473.276238                        2130
2021         Wizards          E  1423.886133                        1320

[825 rows x 4 columns]

### Insight 2 Explanation

Someone may be curious about how good a team is (their elo) and how many people attend each game per season. We will combine the attendance and elo table to see the effect of increasing elo and attendance to see if there is a correlation. It would be reasonable to expect that better teams have higher attendance and this insight would show wheter or not that is true.

In [10]:
def insight3():
    df1 = pd.read_csv("nba_without.csv")
    df1 = df1.drop(["team_elos", "conference"], axis=1)
    team_dict = {"ATL" : "Hawks","BOS" : "Celtics","Van":"Raptors","BRK": "Nets","WAS":"Wizards","CHH" : "Hornets","CHO":"Hornets","CHI" : "Bulls", "CLE" : "Cavaliers","DAL" : "Mavericks","DEN" : "Nuggets","DET": "Pistons","GSW" : "Warriors","HOU" : "Rockets","IND" : "Pacers","LAC" : "Clippers","LAL" : "Lakers","MEM":"Grizzlies","MIA" : "Heat","MIN":"Timberwolves","MIL": "Bucks","NJN": "Nets","BKN" : "Nets","NYK" : "NY Knicks","ORL" : "Magic","PHI" :"76ers","PHO" : "Suns","POR":"Trailblazers","SAC":"Kings","SAS" : "Spurs","SEA": "Thunder","OKC" :"Thunder", "UTA" : "Jazz", "WSB" : "Wizards","NOP" : "Pelicans","TOR":"Raptors"}
    df1 = df1.reset_index()
    df1 = df1.replace({"teams":team_dict})
    df1 = df1.rename(columns={"season":"Season", "teams":"Team"})
    df2 = pd.read_csv("nba_attendance.csv")
    df2 = df2.set_index("Season")
    df2 = df2.drop(["Rank", "Total Home Attendance", "Average Home Attendance", "Home Percent Capacity", "Average Road Attendance", "Road Percent Capacity", "Overall Percent Capacity"], axis=1)
    
    df1 = df1.sort_values(["Season", "Team"])
    df2 = df2.sort_values(["Season", "Team"])
    
    df1 = df1.set_index("Season")
    df2 = df2.drop("Team", axis=1)
    
    newdf = pd.concat((df1, df2), axis=1)
    newdf = newdf.reindex(columns=["Team", "Conference", "game_scores", "Overall Average Attendance"])
    return newdf


############ Function Call ############
insight3()

Team Conference  game_scores Overall Average Attendance
Season                                                                 
1994           76ers          E    97.963415                     14,343
1994           Bucks          E    96.939024                     15,575
1994           Bulls          E    97.489130                     18,327
1994       Cavaliers          E   100.941176                     17,035
1994         Celtics          E   100.817073                     15,420
...              ...        ...          ...                        ...
2021         Thunder          W   105.000000                      1,646
2021    Timberwolves          W   112.125000                      1,996
2021    Trailblazers          W   116.371795                          0
2021        Warriors          W   113.500000                      2,130
2021         Wizards          E   116.329114                      1,320

[825 rows x 4 columns]

### Insight 3 Explanation

Since elo is based off of wins and losses, this insight will allow someone to calculate eastern vs. western elo to see which conference has been stronger throughout the years in terms of their respective wins and losses. This would be able to be compared to the insight 1 table where elos are compared between conferences. The elo calculation in insight 1 are based off of past seasons as well as the current season. However, this insight can be used to determine seasonal elos rather than elos across years. 

In [11]:
def insight4():
    team_dict = {"ATL" : "Hawks","BOS" : "Celtics","Van":"Raptors","BRK": "Nets","WAS":"Wizards","CHH" : "Hornets","CHO":"Hornets","CHI" : "Bulls", "CLE" : "Cavaliers","DAL" : "Mavericks","DEN" : "Nuggets","DET": "Pistons","GSW" : "Warriors","HOU" : "Rockets","IND" : "Pacers","LAC" : "Clippers","LAL" : "Lakers","MEM":"Grizzlies","MIA" : "Heat","MIN":"Timberwolves","MIL": "Bucks","NJN": "Nets","BKN" : "Nets","NYK" : "NY Knicks","ORL" : "Magic","PHI" :"76ers","PHO" : "Suns","POR":"Trailblazers","SAC":"Kings","SAS" : "Spurs","SEA": "Thunder","OKC" :"Thunder", "UTA" : "Jazz", "WSB" : "Wizards","NOP" : "Pelicans","TOR":"Raptors"}
    elo = pd.read_csv("nba_without.csv")
    elo = elo.replace({"teams":team_dict})
    wins = pd.read_csv("nba_record.csv")
    attendance = pd.read_csv("nba_attendance.csv", thousands=',')
    attendance["Overall Average Attendance"].values.astype(int)
    attendance = attendance.filter(items = ["Season", "Team","Overall Average Attendance"])
    wins = wins.drop(["L","GP","Conference"],axis = 1)
    wins = wins.groupby('Season').apply(lambda x: x.loc[x['W'].idxmax(), 'Team'])
    attendance = attendance.groupby('Season').apply(lambda x: x.loc[x['Overall Average Attendance'].idxmax(), 'Team'])
    elo = elo.groupby('season').apply(lambda x: x.loc[x['team_elos'].idxmax(), 'teams'])
    attendance = attendance.reset_index()
    attendance = attendance.drop([0,1,26,27])
    attendance = attendance.reset_index()
    attendance = attendance.drop(["index"],axis = 1)
    attendance = attendance.rename(columns = {0:"Attendance"})
    wins = wins.reset_index()
    wins = wins.rename(columns = {0 : "Wins"})
    elo = elo.reset_index()
    elo = elo.drop([0,1,26,27])
    elo = elo.reset_index()
    elo = elo.drop(["index"],axis = 1)
    elo = elo.rename(columns = {0 : "Elo"})
    df = pd.concat([elo,wins,attendance],axis = 1)
    df["Wins"] = df["Wins"].apply(lambda x : x.split(" ")[1])
    adict = {"Antonio" :"Spurs","Angeles" : "Lakers","State":"Warriors"}
    df = df.replace({"Wins":adict})
    df = df.replace({"Attendance":{"NY Knicks" : "Knicks"}})
    df = df.drop(["Season"],axis = 1)
    df = df.rename(columns = {"season" : "Season"})
    return df






############ Function Call ############
insight4()

Season        Elo       Wins Attendance
0     1996      Bulls      Bulls      Bulls
1     1997      Bulls      Bulls      Bulls
2     1998      Bulls      Spurs      Bulls
3     1999       Jazz     Lakers     Knicks
4     2000     Lakers      Spurs      Bulls
5     2001      Spurs      Kings      Spurs
6     2002     Lakers  Mavericks    Wizards
7     2003  Mavericks     Pacers    Wizards
8     2004      Spurs       Suns    Pistons
9     2005      Spurs    Pistons    Pistons
10    2006    Pistons  Mavericks    Pistons
11    2007  Mavericks    Celtics      Bulls
12    2008      Spurs  Cavaliers    Pistons
13    2009     Lakers  Cavaliers    Pistons
14    2010  Cavaliers      Bulls  Cavaliers
15    2011      Spurs      Bulls      Bulls
16    2012       Heat       Heat      Bulls
17    2013       Heat      Spurs      Bulls
18    2014      Spurs   Warriors      Bulls
19    2015   Warriors   Warriors      Bulls
20    2016   Warriors   Warriors      Bulls
21    2017   Warriors    Rockets      Bulls
22    2018   Warriors      Bucks  Cavaliers
23    2019   Warriors      Bucks   Warriors

### Insight 4 Explanation

Someone might want to know which team performed "best" for each season in each of our datasets. This insight finds the highest eloed team compared for each season to the highest win team for each season and also compared to the highest attendance team for each season. This is reasonable because one metric may be correlated with the other metrics but teams might perform different than expected.

In [12]:
import pandas as pd
import numpy as np
def insight5():
    team_dict = {"ATL" : "Hawks","BOS" : "Celtics","Van":"Raptors","BRK": "Nets","WAS":"Wizards","CHH" : "Hornets","CHO":"Hornets","CHI" : "Bulls", "CLE" : "Cavaliers","DAL" : "Mavericks","DEN" : "Nuggets","DET": "Pistons","GSW" : "Warriors","HOU" : "Rockets","IND" : "Pacers","LAC" : "Clippers","LAL" : "Lakers","MEM":"Grizzlies","MIA" : "Heat","MIN":"Timberwolves","MIL": "Bucks","NJN": "Nets","BKN" : "Nets","NYK" : "NY Knicks","ORL" : "Magic","PHI" :"76ers","PHO" : "Suns","POR":"Trailblazers","SAC":"Kings","SAS" : "Spurs","SEA": "Thunder","OKC" :"Thunder", "UTA" : "Jazz", "WSB" : "Wizards","NOP" : "Pelicans","TOR":"Raptors"}
    wins = pd.read_csv("nba_record.csv")
    attendance = pd.read_csv("nba_attendance.csv", thousands=',')
    attendance["Overall Average Attendance"].values.astype(int)
    attendance = attendance.filter(items = ["Season", "Team","Overall Average Attendance"])
    attendance = attendance.replace({"Team":"Teams"})
    wins = wins.drop(["L","GP","Conference"],axis = 1)
    wint = wins.copy()
    wins = wins.drop([32],axis = 0)
    wint["W"] = wint.groupby(["Season"]).rank(method = "first",ascending = False)
    wins["rank"] = wint["W"]
    wins["Team"] = wins["Team"].apply(lambda x : x.split(" ")[1])
    attendance = attendance.replace({"Team":{"NY Knicks":"Knicks"}})
    wins = wins.replace({"Team":{"Antonio":"Spurs","Bullets":"Wizards","Orleans":"Pelicans","Bobcats":"Hornets","SuperSonics":'Thunder',"Angeles":"Clippers","Jersey":"Nets","York":"Knicks","State":"Warriors","City":"Thunder","Trail":"Trailblazers"}})
    wins = wins.sort_values(by = ["Season","Team"])
    wins = wins.reset_index()
    wins = wins.drop(["index"],axis = 1)
    counter = 1
    for entry in range (0,711):
        counter += 1
        if (wins["Team"][entry] == "Clippers") and (counter % 2 == 1):
            wins["Team"][entry] = "Lakers"
        else:
            continue
    wins = wins.sort_values(["Season","Team"])
    wins = wins.reset_index()
    wins = wins.drop(["index"],axis = 1)
    wins = wins.rename(columns = {"rank" : "Wins Rank"})
    attendancet = attendance.groupby(["Season"]).rank(method = "first",ascending = False)
    attendance["rank"] = attendancet["Overall Average Attendance"]
    attendance = attendance[(attendance["Season"] >= 1996) & (attendance["Season"] <= 2019) ]
    attendance = attendance.sort_values(["Season","Team"])
    attendance = attendance.reset_index()
    attendance = attendance.drop(["index","Team","Season"],axis = 1)
    attendance = attendance.rename(columns = {"rank" : "Attendance Rank"})
    df = pd.concat([wins,attendance],axis = 1)
    df.to_csv("Holder.csv")
    return df



############ Function Call ############
insight5()

<ipython-input-12-7011fdaae02c>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wins["Team"][entry] = "Lakers"


Season          Team   W  Wins Rank  Overall Average Attendance  \
0      1996         76ers  22       25.0                       14798   
1      1996         Bucks  33       20.0                       16179   
2      1996         Bulls  69        1.0                       21778   
3      1996     Cavaliers  42       14.0                       17213   
4      1996       Celtics  15       28.0                       17548   
..      ...           ...  ..        ...                         ...   
706    2019       Thunder  44       10.0                       18034   
707    2019  Timberwolves  19       29.0                       16473   
708    2019  Trailblazers  35       15.0                       18753   
709    2019      Warriors  15       30.0                       19193   
710    2019       Wizards  25       22.0                       17498   

     Attendance Rank  
0               27.0  
1               22.0  
2                1.0  
3               12.0  
4               11.0  
..               ...  
706             12.0  
707             30.0  
708              8.0  
709              1.0  
710             19.0  

[711 rows x 6 columns]

### Insight 5 Explanation

This insight allows the user to compare each team's Wins and their respective wins ranking to their overall average ranking and their respective attendance ranking. This would be useful to see if a particular team's wins rank is correlated to their attendance rank. This data shows that there isn't that much correlation between the two. The attendance for some teams may be skewed because a team that was good the prior season but was impacted by injuries may play poorly but still have high attendance. 

## Data Visualizations

In [20]:
import plotly.express as px
import numpy as np
def visual1():
    df = insight4()
    df = df.replace({"Wins":{"Antonio":"Spurs","Bullets":"Wizards","Orleans":"Pelicans","Bobcats":"Hornets","SuperSonics":'Thunder',"Angeles":"Lakers","Jersey":"Nets","York":"Knicks","State":"Warriors","City":"Thunder","Trail":"Trailblazers"}})
    df = df.replace({"Attendance":{"NY Knicks" : "Knicks"}})
    newdf = df.iloc[:,0:2]
    df = df.drop(["Elo"],axis = 1)
    df = df.rename(columns = {"Wins":"Elo"})
    newdf = pd.concat([newdf,df.iloc[:,0:2]])
    df = df.drop(["Elo"],axis = 1)
    df = df.rename(columns = {"Attendance":"Elo"})
    newdf = pd.concat([newdf,df.iloc[:,0:2]])
    newdf = newdf.reset_index()
    newdf = newdf.groupby(["Elo"]).count()
    newdf = newdf.reset_index()
    newdf = newdf.filter(["Elo","Season"])
    newdf = newdf.rename(columns = {"Elo":"Teams","Season" : "Appearance Count"})
    fig = px.bar(newdf, x="Teams", y="Appearance Count")
    fig.show()
  





############ Function Call ############
visual1()

### Visualization 1 Explanation

We made this visualization to see how often a particular team would be ranked number 1 in elo, wins, or highest overall attendance. From this data we can see that almost half the teams were not ranked top one in the past twenty five years. These graph shows which teams perform the best based off of our metrics. 

In [14]:
def visual2():
    newdf = insight2()
    fig = px.scatter(newdf, x="team_elos", y="Overall Average Attendance",color = "conference" ,trendline = "ols")
    fig.update_yaxes(categoryorder='category ascending')
    fig.show()





############ Function Call ############
visual2()

### Visualization 2 Explanation

This visualization shows a trend that there is not much correlation between how well a team does (team_elos), and their overall average attendance. This is partly due to the fact that some teams are more popular even if they're not playing well, i.e. the Knicks, and larger markets having more popularity than the smaller markets. We also have some outlier points at the bottom of the graph because in the Covid Season, most of the teams hardly had any fans attending their games, if even. 

In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
def visual3():
    df = insight1()
    df = df.reset_index()
    fig = px.line(df, x="Season", y="team_elos", color="Conference")
    fig.update_layout(title="Team ELOs for Each Season by Conference")
    fig.update_yaxes(title_text="Average Team ELOs")
    fig.show()

    fig2 = px.line(df, x="Season", y="W", color="Conference")
    fig2.update_layout(title="Wins for Each Season by Conference")
    fig2.update_yaxes(title_text="Average Wins")
    fig2.show()




############ Function Call ############
visual3()

### Visualization 3 Explanation

This visualization shows a side by side comparison of the average calculated elo for each conference over the seasons and of the average wins for each conference over the seasons. These line graphs show that the Eastern conference performs better for the earlier years than the Western conference, but the West seems to have performed better for both metrics for most other seasons. Both plots show similar patterns but are still very different.

## Summary Files

In [18]:
def summary1():
    team_dict = {"ATL" : "Hawks","BOS" : "Celtics","Van":"Raptors","BRK": "Nets","WAS":"Wizards","CHH" : "Hornets","CHO":"Hornets","CHI" : "Bulls", "CLE" : "Cavaliers","DAL" : "Mavericks","DEN" : "Nuggets","DET": "Pistons","GSW" : "Warriors","HOU" : "Rockets","IND" : "Pacers","LAC" : "Clippers","LAL" : "Lakers","MEM":"Grizzlies","MIA" : "Heat","MIN":"Timberwolves","MIL": "Bucks","NJN": "Nets","BKN" : "Nets","NYK" : "NY Knicks","ORL" : "Magic","PHI" :"76ers","PHO" : "Suns","POR":"Trailblazers","SAC":"Kings","SAS" : "Spurs","SEA": "Thunder","OKC" :"Thunder", "UTA" : "Jazz", "WSB" : "Wizards","NOP" : "Pelicans","TOR":"Raptors"}
    elo = data_parser()
    elo = elo.drop([1994, 1995, 2020, 2021], axis=0)
    elo = elo.reset_index()
    elo = elo.replace({"teams":team_dict})
    elo = elo.sort_values(["season", "teams"])
    elo = elo.rename(columns={"season":"Season", "teams":"Team"})
    elo = elo.set_index("Season")
    attendance = web_parser1()
    attendance = attendance.drop([1994, 1995, 2020, 2021], axis=0)
    attendance = attendance.replace({"Team":{"NY Knicks":"Knicks"}})
    attendance = attendance.sort_values(["Season", "Team"])
    attendance = attendance.drop(["Conference", "Team"], axis=1)
    wins = pd.read_csv("nba_record.csv")
    wins["Team"] = wins["Team"].apply(lambda x : x.split(" ")[1])
    wins = wins.replace({"Team":{"Antonio":"Spurs","Bullets":"Wizards","Orleans":"Pelicans","Bobcats":"Hornets","SuperSonics":'Thunder',"Angeles":"Clippers","Jersey":"Nets","York":"Knicks","State":"Warriors","City":"Thunder","Trail":"Trailblazers"}})
    wins = wins.drop(wins.index[234])
    wins = wins.sort_values(["Season", "Team"])
    wins = wins.set_index("Season")
    wins = wins.drop(["Conference", "Team"], axis=1)
    #return wins
    newdf = pd.concat([elo, attendance, wins], axis=1)
    newdf.to_csv("nba_summary.csv")
    return newdf





############ Function Call ############
summary1()


Team conference    team_elos  game_scores Rank  \
Season                                                           
1996           76ers          E  1282.580640    94.463415   28   
1996           Bucks          E  1378.259560    95.573171   22   
1996           Bulls          E  1744.261468   103.780000    2   
1996       Cavaliers          E  1516.072251    90.729412   11   
1996         Celtics          E  1416.791091   103.597561    9   
...              ...        ...          ...          ...  ...   
2019         Thunder          W  1592.452707   113.942529   15   
2019    Timberwolves          W  1508.709639   112.475610   28   
2019    Trailblazers          W  1604.334756   113.846939    7   
2019        Warriors          W  1661.339935   116.932692    6   
2019         Wizards          E  1457.615812   114.024390   19   

       Total Home Attendance Average Home Attendance Home Percent Capacity  \
Season                                                                       
1996                 489,327                  11,934                    --   
1996                 647,088                  15,782                    --   
1996                 969,149                  23,637                    --   
1996                 730,095                  17,807                    --   
1996                 732,841                  17,874                    --   
...                      ...                     ...                   ...   
2019                 746,323                  18,203                 100.0   
2019                 627,543                  15,305                  80.7   
2019                 799,345                  19,496                 100.3   
2019                 803,436                  19,596                 108.5   
2019                 716,996                  17,487                  86.2   

       Average Road Attendance Road Percent Capacity  \
Season                                                 
1996                    17,661                    --   
1996                    16,577                    --   
1996                    19,919                    --   
1996                    16,620                    --   
1996                    17,222                    --   
...                        ...                   ...   
2019                    17,866                  94.7   
2019                    17,641                  93.7   
2019                    17,992                  96.1   
2019                    18,790                 100.0   
2019                    17,509                  91.8   

       Overall Average Attendance Overall Percent Capacity  GP   W   L  
Season                                                                  
1996                       14,798                       --  82  22  60  
1996                       16,179                       --  82  33  49  
1996                       21,778                       --  82  69  13  
1996                       17,213                       --  82  42  40  
1996                       17,548                       --  82  15  67  
...                           ...                      ...  ..  ..  ..  
2019                       18,034                     97.3  72  44  28  
2019                       16,473                     87.1  64  19  45  
2019                       18,753                     98.2  74  35  39  
2019                       19,193                    104.2  65  15  50  
2019                       17,498                     89.0  72  25  47  

[711 rows x 15 columns]

# Cited Sources

If you used any additional sources to complete your Data Analysis section, list them here:


*   https://plotly.com/python-api-reference/plotly.express.html
*   https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby
*   https://www.w3resource.com/pandas/dataframe/dataframe-rank.php




# Video Presentation

If you uploaded your Video Presentation to Bluejeans, YouTube, or any other streaming services, please provide the link here:


https://bluejeans.com/s/nQr37K1AKHq


Make sure the video sharing permissions are accessible for anyone with the provided link.

# Submission

Prior to submitting your notebook to Gradescope, be sure to <b>run all functions within this file</b>. We will not run your functions ourselves, so we must see your outputs within this file in order to receive full credit.
